In [1]:
# 源码分析
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [2]:
from statsmodels.sandbox.regression.gmm import LinearIVGMM,NonlinearIVGMM
import numpy as np
import pandas as pd

In [3]:
path= 'auto.csv'
data= pd.read_csv(path)
data['constant']=1
exdog_df = data[['gear_ratio','turn','constant']].values
endog_df =  data[['mpg']].values
instrument = data[['gear_ratio','turn','constant']].values

In [4]:
model = LinearIVGMM(endog=endog_df,exog=exdog_df,instrument=instrument)
res1 = model.fit([1,-1,1], maxiter=0) 
print(res1.summary())

                             LinearIVGMM Results                              
Dep. Variable:                      y   Hansen J:                    6.338e-23
Model:                    LinearIVGMM   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Wed, 21 Apr 2021                                         
Time:                        22:34:30                                         
No. Observations:                  74                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.0329      1.502      2.020      0.043       0.090       5.976
x2            -0.7331      0.118     -6.214      0.000      -0.964      -0.502
const         41.2180      8.397      4.909      0.000      24.761      57.675


In [5]:
# 非线性用法 

In [6]:
def moments_1(params, exog):
    beta0,beta1,beta2 = params
    gear,turn,c = exog.T  # unwrap iterable (ndarray)
    # moment condition without instrument    
    err = 1 -  beta0 * gear - beta1 * turn -beta2
    return -err

In [7]:
endog1 = np.zeros(exdog_df.shape[0])  
mod1 = NonlinearIVGMM(endog=endog_df,exog=exdog_df,instrument=instrument,func=moments_1)

In [8]:
w0inv =np.eye(3)
res1 = mod1.fit([0,1,-1], maxiter=2, inv_weights=w0inv)  

         Current function value: 0.000288
         Iterations: 5
         Function evaluations: 582
         Gradient evaluations: 114
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 18
         Function evaluations: 110
         Gradient evaluations: 22


In [9]:
print(res1.summary(yname='y', xname=['gear', 'turn','c']))

                            NonlinearIVGMM Results                            
Dep. Variable:                      y   Hansen J:                    7.540e-09
Model:                 NonlinearIVGMM   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Wed, 21 Apr 2021                                         
Time:                        22:34:30                                         
No. Observations:                  74                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
gear           3.0330      1.502      2.020      0.043       0.090       5.976
turn          -0.7331      0.118     -6.214      0.000      -0.964      -0.502
c             42.2179      8.397      5.028      0.000      25.761      58.675
